In [1]:
import pandas as pd
import tqdm
pd.set_option('display.width', 10000)
filename = '../annotations/geuvadis/Homo_sapiens.GRCh37.050815.cds_introns.gff3'

In [2]:
df = pd.read_csv(filename, sep='\t', skiprows=3, header=None)
df.rename(columns={3:'start_pos', 4:'end_pos', 8:'gene'}, inplace=True)
del df[0]
del df[1]
del df[2]
del df[5]
del df[6]
del df[7]
df.head()

,start_pos,end_pos,gene
0,99884994,99885745,Parent=ENSG00000000003; gene_id=ENSG0000000000...
1,99885874,99887471,Parent=ENSG00000000003; gene_id=ENSG0000000000...
2,99887576,99888391,Parent=ENSG00000000003; gene_id=ENSG0000000000...
3,99888547,99888917,Parent=ENSG00000000003; gene_id=ENSG0000000000...
4,99889037,99890164,Parent=ENSG00000000003; gene_id=ENSG0000000000...


In [3]:
df['intron'] = df['gene'].apply(lambda x: int(x.split('; ')[2].split('_')[2]))
df['gene'] = df['gene'].apply(lambda x: x.split('; ')[1].replace('gene_id=', ''))
df.head()

,start_pos,end_pos,gene,intron
0,99884994,99885745,ENSG00000000003,1
1,99885874,99887471,ENSG00000000003,2
2,99887576,99888391,ENSG00000000003,3
3,99888547,99888917,ENSG00000000003,4
4,99889037,99890164,ENSG00000000003,5


In [4]:
df.shape

(225590, 4)

In [5]:
data = dict()
for w in df.iterrows():
    r = w[1]
    if r[2] not in data:
        data[r[2]] = dict()
    data[r[2]][r[3]] = dict(start_pos=int(r[0]), end_pos=int(r[1]))

import msgpack
import blosc
with open('positions.msg.blosc', 'wb') as f:
    f.write(blosc.compress(msgpack.packb(data)))

In [6]:
with open('positions.msg.blosc', 'rb') as f:
    data = msgpack.unpackb(blosc.decompress(f.read()))